In [131]:
import numpy as np
from time import sleep
import urllib.request
from bs4 import BeautifulSoup
from janome.tokenizer import Tokenizer
import re

#歌詞ページのURLから歌詞本体を取得
def song_scraping(url):
    s_html = urllib.request.urlopen(url)
    s_soup = BeautifulSoup(s_html, "lxml")
    lyric = s_soup.find("div", id="kashi_area")
    
    #HTML上で</br>区切りで改行されているのを用い、正規表現で行ごとの文を取得
    #行ごとに分ける必要がなければ lyric.text でも良い
    lines = re.findall(r'>.*?<', str(lyric))
    lines_cut = []
    for line in lines:
        #何もない行を除く
        if len(line) > 2:
            #空白が"\u3000"になっているので置換
            line = line.replace("\u3000", " ")
            lines_cut.append(line[1:len(line)-1])
    return lines_cut
            
#単語の組み合わせごとの出現頻度を調査
#lines: 歌詞が行ごとにリストで渡される
#dic_id: 今までに登場した単語一覧とそれらのIDの入った辞書
#dic_len: 引数として渡された時点でのdic_idの長さ
#mat: 二次元配列。それぞれの単語の組について、点数を保存している。
#例えば mat[0][3]==1 なら、dic_idでIDが1の単語と3の単語の組み合わせが存在した曲が１曲あったことになる。
#trash: 無視する単語リスト
#block: 単語として意味はあるが無駄に多く登場するので点数をつけない単語リスト
#dic_id, dic_len, mat を更新して返す
def analyze(lines, dic_id, dic_len, mat, trash, block):
    #１つ前、２つ前の単語
    pre = '\0'
    pre2 = '\0'
    #１つ前、２つ前の単語がblockに含まれていないかをここに保存。含まれていたら１が立つ。
    pre_block = [0, 0]
    #この曲で現れた単語の組み合わせはすべてこの二次元配列に保存する。
    mat_temp = np.reshape(np.zeros(len(mat)**2, dtype='uint8'), (len(mat), len(mat)))
    
    t = Tokenizer()
    for line in lines:
        #各行を単語に分ける(形態素解析)
        tokens = t.tokenize(line)  
        for token in tokens:
            #品詞分類
            part_of_speech = token.part_of_speech.split(",")[0]
            part_of_speech_sub = token.part_of_speech.split(",")[1]
            #原形を取得
            base_form = token.base_form


            if (part_of_speech == "名詞" or part_of_speech == "動詞" or part_of_speech == "形容詞" or part_of_speech == "形容動詞")\
                    and part_of_speech_sub != "非自立" and not base_form in trash:
                if not base_form in block:
                    #ここからカウントの対象とみなされた単語に対して行う操作
                    if not base_form in dic_id:
                        #新出の単語ならそれのために dic_id, mat_temp　を拡張
                        dic_id[base_form] = len(dic_id)
                        if len(mat)==0:
                            mat = np.array([[0]], dtype='uint8')
                            mat_temp = np.array([[0]], dtype='uint8')
                        else:
                            mat = np.append(mat, np.reshape([np.zeros(len(mat), dtype='uint8')], (-1, 1)), axis=1)
                            mat = np.append(mat, [np.zeros(len(mat)+1, dtype='uint8')], axis=0)
                            mat_temp = np.append(mat_temp, np.reshape([np.zeros(len(mat_temp), dtype='uint8')], (-1, 1)), axis=1)
                            mat_temp = np.append(mat_temp, [np.zeros(len(mat_temp)+1, dtype='uint8')], axis=0)
                    #単語の組に加点
                    if not pre == '\0' and pre_block[0]==0:
                        i = dic_id[base_form]
                        j = dic_id[pre]
                        if i>j:
                            i, j = j, i
                        mat_temp[i][j] = 1
                    if not pre2 == '\0' and pre_block[1]==0:
                        i = dic_id[base_form]
                        j = dic_id[pre2]
                        if i>j:
                            i, j = j, i
                        mat_temp[i][j] = 1
                    #pre_blockの更新
                    pre_block[1] = pre_block[0]
                    pre_block[0] = 0
                else:
                    pre_block[1] = pre_block[0]
                    pre_block[0] = 1

                pre2 = pre
                pre = base_form
    #この曲で加点された結果を足して上書き
    mat = mat + mat_temp
    return dic_id, len(dic_id), mat

#歌詞と２つの単語を受け取り、その２つの単語の組み合わせを見つけ次第表示する
def show_example(lines, song, artist, word1, word2, trash, block):
    #今何行目にいるか
    now_line = 0
    #１つ目の探すべき単語が直近にあるかどうか。
    #現在見ている単語が探すべき単語なら１、１つ前の単語が探すべき単語なら２、
    #２つ前の単語が探すべき単語なら４を示す。
    flag1 = 0
    #同様に、２つ目の探すべき単語が直近にあるかどうか。
    flag2 = 0
    pre = "\0"
    pre2 = "\0"
    t = Tokenizer()
    for line in lines:
        tokens = t.tokenize(line)
        for token in tokens:
            part_of_speech = token.part_of_speech.split(",")[0]
            part_of_speech_sub = token.part_of_speech.split(",")[1]
            base_form = token.base_form


            if (part_of_speech == "名詞" or part_of_speech == "動詞" or part_of_speech == "形容詞" or part_of_speech == "形容動詞")\
                    and part_of_speech_sub != "非自立" and not base_form in trash and not base_form in block:
                if base_form == word1:
                    flag1 = 1
                if base_form == word2:
                    flag2 = 1
                if flag1+flag2 == 3 or flag1+flag2 == 5:
                    #この場合は見つかったので表示
                    print(song, artist, ':')
                    if now_line == 0:
                        print(lines[now_line])
                    else:
                        print(lines[now_line-1]+lines[now_line])
                    return True
                pre2 = pre
                pre = base_form
                flag1 *= 2
                flag2 *= 2
                flag1 %= 8
                flag2 %= 8
        now_line += 1
    #見つからなかった場合
    return False
                    


            
#曲の一覧のurlが渡される
def scraping(url):
    lyrics = []   
    songs = []
    artists = []
    url_list = []
    for i in range(1,5):
        url_list.append(url + "?p=" + str(i))
    #それぞれのページについて
    for u in url_list:
        print(u)
        html = urllib.request.urlopen(u)
        soup = BeautifulSoup(html, "lxml")
        links = soup.find_all("a")

        for link in links:
            link_content = link.get("href")
            if link_content[0:5] == '/song':
                #曲へのリンクを見つけた場合そこに飛ぶ
                song_url = 'https://www.uta-net.com' + link_content
                lyrics.append(song_scraping(song_url))
                songs.append(link.text)
            elif link_content[0:7] == '/artist':
                #アーティストへのリンクを見つけた場合そのアーティスト名を保存
                artists.append(link.text)
        sleep(0.5)
    return lyrics, songs, artists

#曲のリストから歌詞の１行目を参照して同一と思われる曲を削除
def cut_same_song(lyrics, songs, artists):
    cut = []
    for i in range(len(songs)):
        for j in range(i+1, len(songs)):
            if lyrics[i][0] == lyrics[j][0]:
                cut.append(j)
    cut.sort()
    for k in range(len(cut)):
        del lyrics[cut[k]-k]
        del songs[cut[k]-k]
        del artists[cut[k]-k]
    return lyrics, songs, artists
                



dic_len = 0
dic_id = {}
mat = []

trash = [ "(", ")", "（", "）",  "'", ".", ",", "&", "＆", ":", ";", "!", "?", "…", "-", "！", "？", "a", "m", "s", "t", \
         "I", "me", "you", "You", "we", "We","the", "that", "it", "is", "be", "are", "Are", "to", "ll", "re", "ve", \
         "say", "in", "for", "of", "and", "one", \
         "oh", "yeah", "wow", "baby", "Baby", \
         "さ", "ら", "す", "かな", "かん", \
          "人", "あと", "たち", "もと", "方", "先", "前", "度", "次", "気", "最後", "すべて", "場所", "全て", "的", "底", \
         "れる", "られる", "なる", "する", "いる", "ある", "くる", "いく", "せる", "ない", "いい", \
          "一", "二", "2", "２"]
         
block = ["それ", "そう", "どれ", "どこ", "ここ", "これ", "誰", "何", "なん", \
         "僕", "私", "君", "あなた", "自分", "ぼく", "僕ら", "僕たち", "今", \
         "いま", "いつ", "いつか", "今日", "明日", "日々", \
         "夢",  "幸せ", "気持ち", "恋", "愛", "言葉", "心", "好き", "想い", "未来", "勇気", "笑顔", "道", \
         "思う", "言う", "見る", "行く", "笑う", "愛す", "信じる", \
         "知る", "わかる", "できる", "出来る", "会う", "分かる", "なれる", "見える", \
          "愛しい", "優しい"]

url ="https://www.uta-net.com/user/ranking/monthly.html"

lyrics, songs, artists = scraping(url)
print(len(lyrics))

lyrics, songs, artists = cut_same_song(lyrics, songs, artists)
print(len(lyrics))


for lyric in lyrics:
    dic_id, dic_len, mat = analyze(lyric, dic_id, dic_len, mat, trash, block)
    
    

#print(dic_id)
print()

print(mat)
print(len(mat), len(mat[0]))


for i in range(len(mat)):
    word1 = [key for key, v in dic_id.items() if v==i][0]
    for j in range(i+1, len(mat)):
        word2 = [key for key, v in dic_id.items() if v==j][0]
        if mat[i][j] >= 3:
            print('「' + word1 + '」「' + word2 + '」にかんする' + str(mat[i][j]) + '個の「あるある」が検出されました。')
            print()
            found_num = 0
            for lyric, song, artist in zip(lyrics, songs, artists):
                found = show_example(lyric, song, artist, word1, word2, trash, block)
                if found==True:
                    found_num += 1
                if found_num == 4:
                    break
            print()



1
https://www.uta-net.com/user/ranking/monthly.html?p=1
https://www.uta-net.com/user/ranking/monthly.html?p=2
https://www.uta-net.com/user/ranking/monthly.html?p=3
https://www.uta-net.com/user/ranking/monthly.html?p=4
200
198

[[0. 0. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
5073 5073
「帰る」「無い」にかんする3.0個の「あるある」が検出されました。

いいから WANIMA :
朝帰りメイク落として帰るまるで何も無かったかの様に
CHEEKY WANIMA :
朝まで待てない帰る当て無い
アンビリーバーズ 米津玄師 :
こうしてちゃんと生きてるから 心配いらないよ帰る場所も無く僕らは ずっと向こうまで逃げるんだ

「戻る」「元」にかんする3.0個の「あるある」が検出されました。

ヒューマン WANIMA :
なぁ何処で間違えたんだろう？ゆがみ 元に戻らない
I NEED U (Japanese Ver.) BTS (防弾少年団) :
I need you girlIt goes round &amp; round また元に戻り
愛してるのに、愛せない AAA :
迷いもなく好きだと言えば君の元へ行けるかな戻りたいのに、戻れない

「傷つく」「夜」にかんする3.0個の「あるある」が検出されました。

アウフヘーベン Mrs. GREEN APPLE :
大丈夫 心配無いよ。大勢が傷つくだけなんてことはないよ、直に夜は明ける
手紙 ～拝啓 十五の君へ～ アンジェラ・アキ :
自分の声を信じ歩けばいいの大人の僕も傷ついて眠れない夜はあるけど
One Love 嵐 :
雨の中で君を待ってた 優しさの意味さえ知らずすれ違いに傷ついた夜 それでもここまで来たんだ

「傷つく」「傷つけ

「空」「海」にかんする3.0個の「あるある」が検出されました。

フィクション sumika :
深い海を抜け空飛ぶ街に繰り出し
ハナウタ [ALEXANDROS]×最果タヒ :
影、桃色の空と朝焼けの海、波、まばたき
蜃気楼 Kis-My-Ft2 :
波に溶けながら 逃げていく 蜃気楼やがて空は 満ちる海と 世界を包み込んだ

「空」「飛ぶ」にかんする3.0個の「あるある」が検出されました。

生きていたんだよな あいみょん :
綺麗事だな。精一杯勇気を振り絞って彼女は空を飛んだ
フィクション sumika :
深い海を抜け空飛ぶ街に繰り出し
アンビリーバーズ 米津玄師 :
太陽を見ていた 地面に立ちすくんだままそれでも僕ら 空を飛ぼうと 夢を見て朝を繋いでいく

「窓」「外」にかんする3.0個の「あるある」が検出されました。

灰色と青 ( + 菅田将暉) 米津玄師 :
ぼんやりと背負われたままくしゃみをした窓の外を眺める
スパークル [original ver.] RADWIMPS :
ここでないどこかを 夢見たよ教室の窓の外に
Paper Flower 米津玄師 :
祈るように眠る あなたを見ていた清潔な空気で汚れてしまった 窓の外ブランコが揺れるお庭

「消える」「泣く」にかんする5.0個の「あるある」が検出されました。

vivi 米津玄師 :
「街から子供が消えていく」泣いてるようにも歌を歌う
Paper Flower 米津玄師 :
目の前の思い出が消えていくあの時あなたはなぜ泣いていたの？
I LOVE YOU クリス・ハート :
胸を埋め尽くす不安だけが泣いても 泣いても 消えてくれないの
泣きたいくらい 大原櫻子 :
ずっと大切にしたいから泣きたいくらい

「耳」「澄ます」にかんする4.0個の「あるある」が検出されました。

真赤 My Hair is Bad :
春、恋に落ちて耳を澄まして
Hope 安室奈美恵 :
そっと祈ろう 夜が明けるように目を閉じて耳を澄まして
サヨナラの意味 乃木坂46 :
気配が好きなんだ高架線のその下で耳を澄ましてた
明日晴れるかな 桑田佳祐 :
これ以上元には戻れない耳を澄ませば心の声は

「響く」「歌」にかんする3.0個の「あるある」が検出されました。

小さな恋のうた MONGOL800 :
ほら あなたにと

花束のかわりにメロディーを 清水翔太 :
君だけは守りたいこの手を握ってくれる君

「手」「失う」にかんする3.0個の「あるある」が検出されました。

夜空。feat. ハジ→ miwa :
離れたその瞬間から 心失ってしまったよつないだ手のぬくもり
ガラスを割れ！ 欅坂46 :
(HOUND DOG)すべて失っても手に入れたいものがある
The Beginning ONE OK ROCK :
握りしめた 失わぬようにと…手を広げればこぼれ落ちそうで

「手」「掴める」にかんする3.0個の「あるある」が検出されました。

Nighthawks 米津玄師 :
今はとにかく星が見たい 君の隣で何もないこの手で掴めるのが残りあと一つだけなら
Neo-Aspect Roselia :
灯りは何処へ消えた？ 手繰り寄せるように伸ばす手は何も掴めないまま
Flowerwall 米津玄師 :
離せないんだ もしも手を離せば二度と掴めないような気がして

「時」「流れる」にかんする5.0個の「あるある」が検出されました。

小さな恋のうた MONGOL800 :
小さな恋の思いは届く 小さな島のあなたのもとへあなたと出会い 時は流れる 思いを込めた手紙もふえる
シグナル WANIMA :
顔を洗って 鏡の前 また苦笑い時は流れ 空の下で
HANABI Mr.Children :
笑っていても泣いて過ごしても平等に時は流れる
Don't Leave Me BTS (防弾少年団) :
I wanna know everything時が流れるほど深まる君の

「give」「up」にかんする3.0個の「あるある」が検出されました。

Wake Me Up TWICE :
Up！ Up！ Baby don't give up！ Wake me up！ We can work it out！ YO！
Happiness シェネル :
すぐに分かるよI'm not gonna give up tonight
The Beginning ONE OK ROCK :
a whisper into the nightTelling me it's not my time and don't give up

「can」「out」にかんする3.0個の「あるある」が検出されました。

Wake Me Up